Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [3]:
%pip install azureml-widgets
%pip install xgboost==1.5.0
%pip install --upgrade azureml-sdk[notebooks,automl]
%pip install azureml-train-automl-runtime==1.57.0
%pip install azureml-pipeline-core==1.57.0
%pip install azureml-pipeline-steps

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 16, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 KB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 KB 33.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Not uninstalling urllib3 at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-525bb166-fc19-4c39-bec6-1e96d4e3df1d
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: isodate
    Found existing installation: isodate 0.6.0
    Not uninstalling isodate at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-525bb166-fc19-4c39-bec6-1e96d4e3df1d
    Can't

In [4]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 22, Finished, Available, Finished)

SDK version: 1.57.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [6]:
ws = Workspace.from_config(path="Users/odl_user_268698/starter_files/config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 24, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BJ6C7PZWA to authenticate.
Interactive authentication successfully completed.
quick-starts-ws-268698
aml-quickstarts-268698
southcentralus
610d6e37-4747-4a20-80eb-3aad70a55f43


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [7]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-project-2'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 25, Finished, Available, Finished)

Name,Workspace,Report Page,Docs Page
ml-experiment-project-2,quick-starts-ws-268698,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [8]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 26, Finished, Available, Finished)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [10]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 28, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '69560e40-0cad-43e7-8693-eb39b4ab0d5e'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '69560e40-0cad-43e7-8693-eb39b4ab0d5e', 'run_id': '69560e40-0cad-43e7-8693-eb39b4ab0d5e'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [11]:
dataset.take(5).to_pandas_dataframe()

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 29, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '69560e40-0cad-43e7-8693-eb39b4ab0d5e'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '69560e40-0cad-43e7-8693-eb39b4ab0d5e', 'run_id': '69560e40-0cad-43e7-8693-eb39b4ab0d5e'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [12]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 30, Finished, Available, Finished)

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [13]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 31, Finished, Available, Finished)

Create an AutoMLStep.

In [14]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 32, Finished, Available, Finished)

In [16]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 34, Finished, Available, Finished)

In [17]:
pipeline_run = experiment.submit(pipeline)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 35, Finished, Available, Finished)

Created step automl_module [25d31091][682e4744-694e-4cb2-b21e-17b989e098b4], (This step will run and generate new outputs)
Submitted PipelineRun 64165c3c-3689-4fb6-82b5-2a0ea018938f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/64165c3c-3689-4fb6-82b5-2a0ea018938f?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-268698/workspaces/quick-starts-ws-268698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [18]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 36, Finished, Available, Finished)

Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


Initializing logging file for interpret-community


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [19]:
pipeline_run.wait_for_completion()

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 37, Finished, Available, Finished)

PipelineRunId: 64165c3c-3689-4fb6-82b5-2a0ea018938f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/64165c3c-3689-4fb6-82b5-2a0ea018938f?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-268698/workspaces/quick-starts-ws-268698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 5ec91573-85aa-44c6-b95e-b952254eb911
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5ec91573-85aa-44c6-b95e-b952254eb911?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-268698/workspaces/quick-starts-ws-268698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [20]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 38, Finished, Available, Finished)

Downloaded azureml/5ec91573-85aa-44c6-b95e-b952254eb911/metrics_data, 1 files out of an estimated total of 1


In [21]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 39, Finished, Available, Finished)

,5ec91573-85aa-44c6-b95e-b952254eb911_35,5ec91573-85aa-44c6-b95e-b952254eb911_39,5ec91573-85aa-44c6-b95e-b952254eb911_2,5ec91573-85aa-44c6-b95e-b952254eb911_32,5ec91573-85aa-44c6-b95e-b952254eb911_34,5ec91573-85aa-44c6-b95e-b952254eb911_3,5ec91573-85aa-44c6-b95e-b952254eb911_40,5ec91573-85aa-44c6-b95e-b952254eb911_37,5ec91573-85aa-44c6-b95e-b952254eb911_41,5ec91573-85aa-44c6-b95e-b952254eb911_0,...,5ec91573-85aa-44c6-b95e-b952254eb911_23,5ec91573-85aa-44c6-b95e-b952254eb911_21,5ec91573-85aa-44c6-b95e-b952254eb911_36,5ec91573-85aa-44c6-b95e-b952254eb911_44,5ec91573-85aa-44c6-b95e-b952254eb911_50,5ec91573-85aa-44c6-b95e-b952254eb911_38,5ec91573-85aa-44c6-b95e-b952254eb911_45,5ec91573-85aa-44c6-b95e-b952254eb911_42,5ec91573-85aa-44c6-b95e-b952254eb911_51,5ec91573-85aa-44c6-b95e-b952254eb911_43
precision_score_macro,[0.7705131056255423],[0.7947054718595823],[0.7309221167638172],[0.7672552251941693],[0.7553053408043124],[0.8445288753799393],[0.4440060698027314],[0.8006964705882353],[0.7928045339327563],[0.7781441711329087],...,[0.6546066111041537],[0.7826782947960296],[0.7692516154463418],[0.7818471101431528],[0.8009294558450002],[0.7824325939480865],[0.7771411823188865],[0.7677960038836936],[0.8272454425261289],[0.6748258536933281]
average_precision_score_weighted,[0.9514976507442766],[0.95126721445876],[0.9290388597667807],[0.9454733030627387],[0.9383937840791968],[0.9340512128892066],[0.8011068409624182],[0.9476170311555095],[0.9506077780819907],[0.9535822261506017],...,[0.9365201551326937],[0.9512078722916547],[0.9482474801208366],[0.9525415198593011],[0.9562956265309095],[0.953909745885795],[0.9503754598178983],[0.9489569984923903],[0.956199350130121],[0.9443374593705292]
recall_score_macro,[0.7298299332959153],[0.690094137783483],[0.5840974387187481],[0.7570649647029621],[0.6666930630345265],[0.5052491724507129],[0.5],[0.6479539573249458],[0.7494956904456263],[0.7445642005975768],...,[0.8361679327661355],[0.7281572371431164],[0.7125685610923095],[0.7464318593971996],[0.7383378994418789],[0.7538784136986961],[0.733394369145332],[0.7426845013494564],[0.6256263348689536],[0.8677833719553874]
average_precision_score_macro,[0.8107914567139332],[0.8125343827284399],[0.7300395466588593],[0.7877037417412831],[0.7623733086913668],[0.7495381136746629],[0.5],[0.8040998303278059],[0.8116281263732896],[0.8171134643523112],...,[0.7592478670030633],[0.8105653493151015],[0.7973949197161567],[0.812931405983517],[0.8279091613294299],[0.8190021212604568],[0.8051317764002468],[0.7997432437434344],[0.8275316656990448],[0.7870006929600559]
precision_score_weighted,[0.9008132024091846],[0.8986248389164687],[0.8663154637657997],[0.9054031432088335],[0.8859481117753807],[0.8790843638007296],[0.788565560086672],[0.892533922699277],[0.9091673500117678],[0.9051980543721705],...,[0.9116181731423262],[0.9030335205195396],[0.8973044160136986],[0.9063183051203674],[0.9087639465171337],[0.9078672082243358],[0.9028645680005167],[0.9027129543017738],[0.8945945932093288],[0.921129786749879]
f1_score_micro,[0.9071320182094081],[0.9101669195751139],[0.8922610015174507],[0.9071320182094081],[0.9001517450682853],[0.8889226100151745],[0.8880121396054628],[0.9068285280728376],[0.9147192716236723],[0.9101669195751139],...,[0.7763277693474963],[0.9104704097116844],[0.9059180576631259],[0.9113808801213961],[0.9159332321699545],[0.9119878603945372],[0.9092564491654022],[0.9068285280728376],[0.9071320182094081],[0.8030349013657057]
log_loss,[0.18573013603201033],[0.6256086252570332],[0.2414513116338928],[0.2516771195634219],[0.2261469353637422],[0.24429183787704353],[0.4100739713986146],[0.675331759773324],[0.4415669171516007],[0.17851374134751752],...,[0.4527478269759932],[0.2258737328633013],[0.19353624173211753],[0.18071078454259476],[0.19336006883744952],[0.18235598889252908],[0.18725096769447408],[0.2239286879389177],[0.21608928449505657],[0.3732403451583006]
AUC_weighted,[0.938711338582969],[0.935889242692837],[0.898600899884597],[0.9314750290360047],[0.921725970506

### Retrieve the Best Model

In [22]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 40, Finished, Available, Finished)

Downloaded azureml/5ec91573-85aa-44c6-b95e-b952254eb911/model_data, 1 files out of an estimated total of 1


In [23]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 41, Finished, Available, Finished)

[08:58:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[08:58:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[08:58:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/synfs/notebook/0/aml_notebook_mount')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('31', Pipeline(steps=[('standardscalerwrappe...ambda=0.10416666666666667, subsample=0.7, tree_method='auto'))]))], flatten_transform=False, weights=[0.14285714285714285, 0.07142857142857142, 0.21428571428571427, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [24]:
best_model.steps

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 42, Finished, Available, Finished)

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.4, gamma=5, max_depth=6, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.7708333333333335, reg_lambda=1.5625, subsample=0.5, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclas

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [25]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 43, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '69560e40-0cad-43e7-8693-eb39b4ab0d5e'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '69560e40-0cad-43e7-8693-eb39b4ab0d5e', 'run_id': '69560e40-0cad-43e7-8693-eb39b4ab0d5e'}


#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [26]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

StatementMeta(ef21626c-69c8-45dc-850a-aff25dab2560, 0, 44, Finished, Available, Finished)

PredictionException: PredictionException:
	Message: 'XGBModel' object has no attribute 'callbacks'
	InnerException: AttributeError: 'XGBModel' object has no attribute 'callbacks'
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Encountered an internal AutoML error. Error Message/Code: PredictionException. Additional Info: PredictionException:\n\tMessage: 'XGBModel' object has no attribute 'callbacks'\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"message\": \"'XGBModel' object has no attribute 'callbacks'\",\n        \"target\": \"Xgboost\",\n        \"reference_code\": \"Xgboost\"\n    }\n}",
        "details_uri": "https://aka.ms/automltroubleshoot",
        "target": "Xgboost",
        "inner_error": {
            "code": "ClientError",
            "inner_error": {
                "code": "AutoMLInternal"
            }
        },
        "reference_code": "Xgboost"
    }
}

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()